In [14]:
from pymongo import MongoClient
import requests

def toInt(s):
    try: 
        return int(s)
    except ValueError:
        return s

def getDictFromText(url,id):
    r = requests.get(url, 'r')
    dataArray = [line.split('\t') for line in r.text.splitlines()]
    colArray = dataArray.pop(0)
    result={toInt(data[id]) : {col : toInt(data[idx]) for idx, col in enumerate(colArray)}for data in dataArray}
    return result

def getListFromText(url):
    r = requests.get(url, 'r')
    dataArray = [line.split('\t') for line in r.text.splitlines()]
    colArray = dataArray.pop(0)
    result=[{col : toInt(data[idx]) for idx, col in enumerate(colArray)}for data in dataArray]
    return result

client = MongoClient('ec2-52-211-76-129.eu-west-1.compute.amazonaws.com',27017)
db = client.wax_dfg
db.departement.drop()
urlRegion= 'http://www.insee.fr/fr/methodes/nomenclatures/cog/telechargement/2016/txt/reg2016.txt'
dictRegion = getDictFromText(urlRegion,0)

urlDpt= 'http://www.insee.fr/fr/methodes/nomenclatures/cog/telechargement/2016/txt/depts2016.txt'
listDpt = getListFromText(urlDpt)
for dpt in listDpt:
    dpt['REGION'] = dictRegion[dpt['REGION']]
    if dpt['DEP'] == '2A':
        dpt['DEP'] = 20
db.departement.insert_many(listDpt)

